In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import scipy.sparse as ssp
from sklearn import metrics, model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def read_text_data(filename):
    texts = []
    with open(filename) as fi:
        fi.readline()
        for line in fi:
            id, text = line.split("||")
    texts.append(text)
    return texts

In [ ]:
path = "../input/"


In [ ]:
train_texts = read_text_data(path+"training_text")
test_texts = read_text_data(path+"test_text")


In [ ]:
tfidf = TfidfVectorizer(
    min_df=1, max_features=1600, strip_accents='unicode', lowercase =True,
    analyzer='word', token_pattern=r'\w+', ngram_range=(1, 3), use_idf=True,
    smooth_idf=True, sublinear_tf=True, stop_words = 'english'
).fit(train_texts)

In [ ]:
X_train_text = tfidf.transform(train_texts)
X_test_text = tfidf.transform(test_texts)


In [ ]:
X_train_text

In [ ]:
train = pd.read_csv(path+"training_variants")
test = pd.read_csv(path+"test_variants")


In [ ]:
ID_train = train.ID
ID_test = test.ID

In [ ]:
y = train.Class.values-1

In [ ]:
train.head()

In [ ]:
train = train.drop(['ID','Class'], axis=1)
test = test.drop(['ID'], axis=1)

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
data = train.append(test)

In [ ]:
data.shape

In [ ]:
X_data = pd.get_dummies(data).values
X_data.shape

In [ ]:
X = X_data[:train.shape[0]]
X_test = X_data[train.shape[0]:]


In [ ]:
X.shape


In [ ]:
X_test.shape


In [ ]:

X_test_text.shape


In [ ]:
X_train_text.shape


In [ ]:
X = ssp.hstack((X_train_text, X), format='csr')
X_test = ssp.hstack((X_test_text, X_test), format='csr')

In [ ]:
print(X.shape, X_test.shape)

In [ ]:
y_test = np.zeros((X_test.shape[0], max(y)+1))


In [ ]:
n_folds = 5

In [ ]:
kf = model_selection.StratifiedKFold(n_splits=n_folds, random_state=1, shuffle=True)

In [ ]:
fold = 0
for train_index, test_index in kf.split(X, y):
    fold += 1

In [ ]:
X_train, X_valid    = X[train_index], 	X[test_index]
y_train, y_valid    = y[train_index],   y[test_index]

print("Fold", fold, X_train.shape, X_valid.shape)

In [ ]:
clf = LogisticRegression(C=3)

clf.fit(X_train, y_train)

In [ ]:
p_train = clf.predict_proba(X_train)
p_valid = clf.predict_proba(X_valid)
p_test = clf.predict_proba(X_test)


In [ ]:
print(metrics.log_loss(y_train, p_train))
print(metrics.log_loss(y_valid, p_valid))

In [ ]:
y_test += p_test/n_folds

In [ ]:
classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
subm = pd.DataFrame(y_test, columns=classes)
subm['ID'] = ID_test
subm.to_csv('model_2.csv', index=False)


In [ ]:
subm.head()